<a href="https://colab.research.google.com/github/yaobviously/sym-cargo/blob/main/sym_cargo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install plotly-express --quiet
!pip install vincenty --quiet

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from vincenty import vincenty
from sklearn.neighbors import NearestNeighbors
import plotly_express as px

In [4]:
port_file = '/content/drive/MyDrive/Ports/ports.csv'
tracking_file = '/content/drive/MyDrive/Ports/tracking.csv'

In [126]:
def wrangle():
  df1 = pd.read_csv(port_file)

  # converting lat and long to radians to compute haversine distance
  df1['lat_rad'] = np.radians(df1['lat'])
  df1['long_rad'] = np.radians(df1['long'])
  
  # rounding lat and long in port df
  df1['lat'] = df1['lat'].round(1)
  df1['long'] = df1['long'].round(1)

  df2 = pd.read_csv(tracking_file, parse_dates=['datetime'])
  df2 = df2.drop_duplicates()
  df2 = df2.sort_values(['vessel', 'datetime'])
  
  # converting lat and long to radians to compute haversine distance  
  df2['lat_rad'] = np.radians(df2['lat'])
  df2['long_rad'] = np.radians(df2['long'])

  # adding lat/long column and lat/long 1 back to later compute delta
  df2['lat_long'] = [[x,y] for x, y in zip(df2['lat'], df2['long'])]
  df2['lat_long_1back'] = df2.groupby('vessel')['lat_long'].transform(lambda x: x.shift())   
  
  # months and days
  df2['month'] = df2['datetime'].dt.month
  df2['day'] = df2['datetime'].dt.day
  df2['time_delta'] = df2.groupby('vessel')['datetime'].transform(lambda x: x.diff(1))
  df2['hours'] = df2.groupby('vessel')['time_delta'].transform(lambda x: [(n.seconds/3600) + (n.days *24) for n in x])

  # speed and heading back
  df2['speed_diff_2back'] = df2.groupby('vessel')['speed'].transform(lambda x: x.diff(2))
  df2['speed_diff_1back'] = df2.groupby('vessel')['speed'].transform(lambda x: x.diff(1))
  df2['speed_diff_1ahead'] = df2.groupby('vessel')['speed'].transform(lambda x: x.diff(-1))
  df2['speed_diff_2ahead'] = df2.groupby('vessel')['speed'].transform(lambda x: x.diff(-2))
  df2['heading_diff_2back'] = df2.groupby('vessel')['heading'].transform(lambda x: x.diff(2))
  df2['heading_diff_1back'] = df2.groupby('vessel')['heading'].transform(lambda x: x.diff(1))
  df2['heading_diff_1ahead'] = df2.groupby('vessel')['heading'].transform(lambda x: x.diff(-1))
  df2['heading_diff_2ahead'] = df2.groupby('vessel')['heading'].transform(lambda x: x.diff(-2))
  
  df2['vessel_1back'] = df2['vessel'].shift(1)
  df2 = df2.reset_index(drop=True)

  return df1, df2

In [146]:
df1, df2 = wrangle()

In [46]:
# creating a dictionary of ports with their lat and longs - will be used to
# repeatedly to explore data and assign values

ports = {port:(lat, long) for port, lat, long in zip(df1['port'], df1['lat'], df1['long'])}

# a dictionary to retrieve the port id from the index
idx_ports = {idx:port for idx, port in zip(df1.index, df1.port)}

In [ ]:
# training a nearest neighbor model to  find the closest port when the 
# conditions indicating an extended stop have occurred. the metric is haversine
# in order to compute the 'great circle' distance. so i don't forget, the model
# returns the *index* of the port, not the port's identifying label

ports_train = df1[['lat_rad', 'long_rad']]

neigh_ports = NearestNeighbors(n_neighbors=3, algorithm='ball_tree', metric='haversine')
neigh_ports.fit(ports_train)

dist, n = neigh_ports.kneighbors(np.array([0.677565, 0.469731]).reshape(1,-1))

print(dist[0] * 6370)
print([idx_ports[n] for n in n[0]])

In [20]:
# using set differences i noticed two ports (42, 113) were never being assigned,
# so i used vincenty distance to figure out what was going on. since the lat/long
# of where the boats are docked are, in both cases, much closer to the port
# assigned by the model, i cannot justify any change

coord_first_stop = (38.743843, 26.895678)               
coord_second_stop = (38.821700, 26.913800)

viable_ports = [82, 113]

vin_one = {f'port{a}':[vincenty(ports[a], coord_first_stop)] for a in viable_ports}
vin_two = {f'port{a}':[vincenty(ports[a], coord_second_stop)] for a in viable_ports}

In [21]:
# there are five ports in Japan that are very close to each other. note ports
# 42 and 51 are the same port - they have precisely the same distances from two
# different docking locations.

jports = [42, 109, 65, 51, 30]

jc1 = (35.470100, 139.738700)    # has a predicted distance of 0.000022 from 51
jc2 = (35.461105, 139.718472)    # has a predicted distance of 0.000347 from 51

jc1_dist = {f'port_{a}':[vincenty(ports[a], jc1)] for a in jports}
jc2_dist = {f'port_{a}':[vincenty(ports[a], jc2)] for a in jports}

In [22]:
# functions to quickly inspect dataframe to figure out what's up

def get_dates(vessel=111, start='2019-01-01', end='2019-12-25'):

  df = df2[(df2.datetime >= start) & (df2.datetime <=end)]
  df = df[df['vessel'] == vessel]
  
  return df

def where_zero(vessel = 75):
  
  df_ = df2[(df2['vessel'] == vessel) & (df2['speed'] <= 0.5)]
  
  return df_

In [23]:
def nearest_port(df):
  """
  a simple function to return the port (identifier) of the nearest port in the
  dataset. 
  """

  data = np.array([df['lat_rad'], df['long_rad']]).reshape(1, -1)
  dist, pred = neigh_ports.radius_neighbors(data, radius=0.006, sort_results = True) 

  if len(dist[0]) == 0:
    return -1
  
  else:
    return idx_ports[pred[0][0]]

In [24]:
def nearest_distance(df):
  """
  a simple function to return the distance of the nearest port in the dataset
  """
  
  data = np.array([df['lat_rad'], df['long_rad']]).reshape(1, -1)
  dist, pred = neigh_ports.radius_neighbors(data, radius=0.0075, sort_results = True) 

  if len(dist[0]) == 0:
    return -1
  
  else:
    return dist[0][0]


In [51]:
def vincent_distance(row):
  """ 
  a simple function that returns the vincenty distance for lat and longitude
  coordinate pairs
  """
  if row['vessel'] != row['vessel_1back']:
    return -99

  loc1 = row['lat_long']
  loc2 = row['lat_long_1back']

  try:
    distance = vincenty(loc1, loc2)
    return distance
  except:
    return -99

In [147]:
# using my simple neighbor model with haversine distance to assign the closest 
# port along with its distance to the dataframe. i used a mask to speed up
# the process

mask = ((df2['speed'] <= 0.2) & \
        ((abs(df2['heading_diff_1back'] <= 2)) & (abs(df2['heading_diff_2back'] <=2)) | \
         (abs(df2['heading_diff_1ahead'] <=2)) & (abs(df2['heading_diff_2ahead'] <=2)))
)
  
df_valid = df2[mask]

df2['pred_port'] = -1
df2['pred_dist'] = -1
df2.loc[mask, 'pred_port'] = df_valid.apply(nearest_port, axis=1)
df2.loc[mask, 'pred_dist'] = df_valid.apply(nearest_distance, axis=1)

In [148]:
# calculating the vincenty distances between each row in the dataframe to
# detect movement. this seems like it might allow me to more easily detect when
# a ship is in japanese (and other?) ports where they appear to unload/load 
# without zero movement 

vin_mask = df2['speed'] <= 1

df_vin = df2[vin_mask]

df2['vin_delta'] = -99
df2.loc[vin_mask, 'vin_delta'] = df_vin.apply(vincent_distance, axis=1)

In [149]:
df2['pred_port_coords'] = [ports[k] if k in ports else -99 for k in df2['pred_port']]

In [143]:
def vincenty_port(row):

  if row['vessel'] != row['vessel_1back']:
    return -99

  try:
    distance = vincenty(row['pred_port_coords'], row['lat_long'])
    return distance
  except:
    return -99

In [150]:
df2['vindist_pred_port'] = df2.apply(vincenty_port, axis=1)

In [ ]:
# the next few cells are a deep dive on the ports that aren't visited 

port_60 = [31.6940667, 121.7672667]
port_88 = [21.48393, 109.0811]
port_109 = [35.49415, 139.7681333]
port_113 = [38.45433, 27.1685]

distances_from_60 = {vessel:vincenty(port_60, d) for vessel, d in zip(df2.index, df2['lat_long'])}
distances_from_88 = {vessel:vincenty(port_88, d) for vessel, d in zip(df2.index, df2['lat_long'])}          # returning a none for some reason - manually delete using loop
distances_from_109 = {vessel:vincenty(port_109, d) for vessel, d in zip(df2.index, df2['lat_long'])}
distances_from_113 = {vessel:vincenty(port_113, d) for vessel, d in zip(df2.index, df2['lat_long'])}

sorted_60 = sorted(distances_from_60.items(), key = lambda x: x[1], reverse=False)[:10]
# sorted_88 = sorted(distances_from_88.items(), key = lambda x: x[1], reverse=False)[:10]
sorted_109 = sorted(distances_from_109.items(), key = lambda x: x[1], reverse=False)[:10]
sorted_113 = sorted(distances_from_113.items(), key = lambda x: x[1], reverse=False)[:10]

In [ ]:
# looking more carefully at the nearest neighbor ports for Japan using vincenty
# distance. 42 doesn't require inclusion.

japan = df2[df2['pred_port'].isin([109, 51, 65, 30])]
jp_port = japan[japan['pred_port'] == 51]

port_51 = 35.4700833, 139.7402667
port_65 = 35.4736489, 139.9677229
port_30 = 35.3421341, 139.8226354

jp_port['dist_109'] = [vincenty(port_109, x) for x in jp_port['lat_long']]
jp_port['dist_51'] = [vincenty(port_51, x) for x in jp_port['lat_long']]
jp_port['dist_65'] = [vincenty(port_65, x) for x in jp_port['lat_long']]
jp_port['dist_30'] = [vincenty(port_30, x) for x in jp_port['lat_long']]

In [151]:
# creating a dictionary of dataframes for each vessel

vessel_dfs = {}

for vessel in df2.vessel.unique():
  df_ = df2[df2['vessel'] == vessel]
  vessel_dfs[vessel] = df_.set_index('datetime')

In [152]:
# creating a dictionary of dataframes for each port

port_ds = {}

for port in df2.pred_port.unique():
  df_ = df2[df2['pred_port'] == port]
  port_ds[port] = df_.set_index('datetime')

del port_ds[-1]

for n in port_ds.keys():
  port_ds[n][f'{n}_coord'] = [ports[n]] * len(port_ds[n])

for n in port_ds.keys():
  port_ds[n]['vdist_port'] = [vincenty(x, y) for x, y in zip(port_ds[n][f'{n}_coord'], port_ds[n]['lat_long'])]


In [ ]:
for p in port_ds.keys():
  print(f'{p}:', len(port_ds[p]))

In [160]:
for v in vessel_dfs.keys():
  print(f'{v}:', len(vessel_dfs[v]))

1: 3905
2: 4078
3: 3828
4: 4016
5: 4237
6: 3651
7: 4108
8: 3915
9: 3669
10: 4014
11: 4206
12: 4271
13: 4199
14: 3507
15: 3853
16: 3574
17: 3583
18: 3740
19: 4348
20: 4099
21: 3712
22: 3512
23: 4071
24: 3909
25: 4118
26: 3977
27: 4017
28: 3979
29: 4160
30: 3946
31: 4014
32: 4104
33: 3849
34: 3790
35: 3531
36: 4062
37: 4278
38: 3860
39: 4073
40: 3938
41: 3802
42: 3402
43: 4102
44: 3646
45: 3976
46: 2817
47: 3877
48: 3406
49: 4163
50: 3368
51: 3269
52: 3460
53: 4162
54: 2986
55: 4251
56: 4214
57: 3892
58: 4321
59: 4311
60: 3199
61: 4109
62: 3748
63: 4199
64: 2912
65: 3869
66: 1456
67: 3925
68: 3766
69: 3933
70: 3859
71: 4063
72: 3973
73: 4106
74: 4116
75: 1215
76: 3408
77: 4045
78: 3424
79: 3607
81: 3474
82: 3908
83: 3498
84: 3075
85: 1701
86: 2112
87: 3810
88: 3772
89: 3919
90: 4090
91: 4332
92: 2940
93: 4097
94: 3429
95: 3410
96: 3915
97: 3487
98: 3060
99: 2889
100: 4104
101: 3418
102: 3034
103: 3606
104: 4163
105: 3747
106: 3911
107: 1523
108: 3839
109: 4249
110: 3645
111: 3669
112: 35

In [ ]:
# a cell to check distances

port_63 = ports[63]
d_from_63 = [[v, vincenty(port_63, d)] for v, d in zip(df2['vessel'], df2['lat_long'])]
d63 = pd.DataFrame.from_records(d_from_63, columns = ['vessel', 'distance'])
d63.groupby('vessel')['distance'].min().sort_values().head(30)

In [62]:
maxes = {}

for n in port_ds.keys():
  max_ = port_ds[n]['vdist_port'].max()
  maxes[n] = max_

In [63]:
mins = {}

for n in port_ds.keys():
  min_ = port_ds[n]['vdist_port'].min()
  minb = port_ds[n]['pred_dist'].min() * 1000
  mins[n] = (min_, minb)

In [176]:
nz.pred_port[3]

152

In [181]:
df66 = vessel_dfs[66]

nz = df66[df66['pred_port'] > 0]

zipped = zip(nz.index, nz.pred_port, nz.vindist_pred_port)

starts = list

for i in range(len(nz.pred_port)):
  first = 0
  last = 0
  if nz.pred_port[i] == nz.pred_port[i+1]:
    pass
  else:
    last = i
    starts.append([nz.index[first], nz.pred_port[i], nz.index[last]])
    starts = i + 1


TypeError: ignored

In [171]:
print(vessel_dfs[66][['lat_long', 'pred_port', 'pred_dist', 'vin_delta', 'vindist_pred_port']].to_string())

                                                       lat_long  pred_port  pred_dist   vin_delta  vindist_pred_port
datetime                                                                                                            
2019-06-20 02:22:29                     [34.681438, 136.545213]         -1  -1.000000  -99.000000         -99.000000
2019-06-21 23:48:09                     [34.681437, 136.545283]         -1  -1.000000    0.006416         -99.000000
2019-06-22 00:12:09              [34.68138, 136.54523500000002]        152   0.005068    0.007703          35.700112
2019-06-22 03:36:12                     [34.681393, 136.545252]        152   0.005068    0.002123          35.698466
2019-06-22 14:27:19                      [34.68143, 136.545272]         -1  -1.000000    0.004495         -99.000000
2019-06-22 17:39:12             [34.682888, 136.54585500000002]         -1  -1.000000  -99.000000         -99.000000
2019-06-22 20:48:38                     [34.706598, 136.583107] 

In [68]:
# simple mapping tool to inspect routes

fig = px.scatter_geo(f,lat='lat',lon='long', hover_name="datetime")

fig.update_layout(title = 'Vessel Locations', title_x=0.5)
fig.show()

In [ ]:
# the dataframes to submit

voyages = pd.DataFrame(columns= ['vessel', 'begin_date', 'end_date', 'begin_port_id', 'end_port_id'])
predict = pd.DataFrame(columns= ['vessel', 'begin_port_id', 'end_port_id', 'voyage'])